# Flower classification with resnet

License: public domain.

This is based on the material presented in [Lesson 1: Image classification](https://course.fast.ai/videos/?lesson=1) and [Lesson 2: Data cleaning and production; SGD from scratch](https://course.fast.ai/videos/?lesson=2) (the first half) of the [Practical Deep Learning for Coders, v3](https://course.fast.ai) course by fast.ai.

## Setup

Make sure the code is reloaded automatically and plots are displayed inline.

In [ ]:
%reload_ext autoreload
%autoreload 2
%matplotlib inline

Import the fast.ai libraries.

In [ ]:
from fastai.vision import *
from fastai.metrics import error_rate

## Looking at the data

Use the [Flower Color Images](https://www.kaggle.com/olgabelitskaya/flower-color-images) dataset published by Olga Belitskaya.

> The content is very simple: 210 images (128x128x3) with 10 species of flowering plants and the file with labels flower-labels.csv. Photo files are in the .png format and the labels are the integers.

In [ ]:
LABEL_MAP = {
    0: 'phlox',
    1: 'rose',
    2: 'calendula',
    3: 'iris',
    4: 'leucanthemum maximum',
    5: 'bellflower',
    6: 'viola',
    7: 'rudbeckia laciniata',
    8: 'peony',
    9: 'aquilegia' 
}

# Need to set this to a writable directory.
MODEL_DIR = '/kaggle/working'
DATA_DIR = '../input/flower_images/flower_images'
LABELS = 'flower_labels.csv'
IMAGE_SIZE = 204  # decreasing this negatively affects the results

df = pd.read_csv(f'{DATA_DIR}/{LABELS}')
df.replace(to_replace=LABEL_MAP, inplace=True)
df.head()

In [ ]:
tfms = get_transforms(do_flip=False)
data = ImageDataBunch.from_df(DATA_DIR, df, ds_tfms=tfms, size=IMAGE_SIZE)
data.classes

In [ ]:
data.show_batch(rows=5, figsize=(12, 12))

In [ ]:
len(data.classes), data.c

## Training: resnet34

We will use a [convolutional neural network](http://cs231n.github.io/convolutional-networks), which will be trained for 4 epochs (4 cycles through all the data).

In [ ]:
learn = cnn_learner(data, models.resnet34, metrics=error_rate)

In [ ]:
learn.model

In [ ]:
learn.fit_one_cycle(4)

In [ ]:
learn.model_dir = MODEL_DIR
learn.save('stage-1')

### Results

In [ ]:
interp = ClassificationInterpretation.from_learner(learn)
losses, idxs = interp.top_losses()
len(data.valid_ds) == len(losses) == len(idxs)

In [ ]:
interp.plot_top_losses(9, figsize=(15, 11))

In [ ]:
interp.plot_confusion_matrix(figsize=(12, 12), dpi=60)

This is not good: lots of mispredictions.

In [ ]:
interp.most_confused(min_val=1)

### Model tuning

So far, we've been using the model exactly as is.  We've added a few layers to the network and only trained those. It's fast to train and works reasonably well if the network is trained on similar data. See the [Visualizing and Understanding Convolutional Networks](https://cs.nyu.edu/~fergus/papers/zeilerECCV2014.pdf) paper by Matthew D. Zeiler and Rob Fergus to understand why.

So let's allow to train the whole network by calling `unfreeze` and compare the error with the previous result.

In [ ]:
learn.unfreeze()
learn.fit_one_cycle(1)

Now let's revert back to the previous model and look at the learning rate.

In [ ]:
learn.load('stage-1');  # ; omits the output

In [ ]:
learn.lr_find()

In [ ]:
learn.recorder.plot()

Looking at the plot, pick the learning rate range before the loss starts getting worse and train the whole model.

In [ ]:
learn.unfreeze()
learn.fit_one_cycle(2, max_lr=slice(1e-6, 1e-2))

Compare the error with the previous result.

Now let's plot the confusion matrix.

In [ ]:
interp = ClassificationInterpretation.from_learner(learn)
interp.plot_confusion_matrix(figsize=(12, 12), dpi=60)

The model has improved, but there are still misses.

## Training: resnet50

Let's see if a different network architecture produces better results. See the [Deep Residual Learning for Image Recognition](https://arxiv.org/pdf/1512.03385.pdf) paper by Kaiming He, Xiangyu Zhang, Shaoqing Ren, and Jian Sun.

In [ ]:
data = ImageDataBunch.from_df(DATA_DIR, df, ds_tfms=tfms, size=IMAGE_SIZE, bs=64).normalize(imagenet_stats)
learn = cnn_learner(data, models.resnet50, metrics=error_rate)
learn.model_dir = MODEL_DIR
learn.lr_find()

In [ ]:
learn.recorder.plot()

In [ ]:
learn.fit_one_cycle(8)

In [ ]:
learn.save('stage-1-50')

In [ ]:
interp = ClassificationInterpretation.from_learner(learn)
interp.plot_confusion_matrix(figsize=(12, 12), dpi=60)

Much better results this time.

In [ ]:
interp.most_confused(min_val=1)

In [ ]:
interp.plot_top_losses(4, figsize=(10, 10))

Let's see if it's possible to improve the model even further based on the plotted learning rate.

In [ ]:
learn.unfreeze()
learn.fit_one_cycle(3, max_lr=slice(1e-6, 1e-3))

In [ ]:
interp = ClassificationInterpretation.from_learner(learn)
interp.plot_confusion_matrix(figsize=(12, 12), dpi=60)

In [ ]:
interp.most_confused(min_val=1)

## Exporting the model

Let's export the model so it can be used in production.

In [ ]:
learn.path = Path(MODEL_DIR)
PKL_FILE = Path('resnet-flowers.pkl')
learn.export(file=PKL_FILE)
!ls $MODEL_DIR/$PKL_FILE

After committing the kernel, the model will be available in the Output section.